In [52]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score

In [2]:
data = pd.read_csv('train.csv')
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [11]:
#We will check the shape of the data frame
data.shape

(20800, 5)

In [10]:
# check how many values are none and we have to drop it.
data.isnull().sum(axis=0)

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [12]:
df = data.dropna()

In [13]:
df.shape

(18285, 5)

In [14]:
#we have deleted all the data which had NAN values becasue it can affect the accuracy.

In [15]:
# get the independent and dependent variables
x = df.drop('label',axis=1)
y = df['label']

In [16]:
x.shape

(18285, 4)

In [17]:
y.shape

(18285,)

In [18]:
import tensorflow
from tensorflow.keras.layers import Embedding,LSTM,Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential

In [19]:
#take a vocabulary size 
vocab_size = 5000

# One hot representation of data

In [20]:
# As we dropped the values we have to reset the index to make one hot representation of data
sentences = x.copy()
sentences.reset_index(inplace=True)

In [25]:
# Now, we have to preprocess the text because we have many ounctuations and STOPWORDS in our dataset so we have to preprocess that data
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [24]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\baps\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(sentences)):
    review = re.sub('[^a-zA-Z]', ' ', sentences['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [28]:
#here we can see that corpus contains the words after preprocessing done.
corpus[:10]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag']

In [30]:
# We can see that the words in corpus are represented in onehot encode w.r.t. given vocab_size that is 5000
onehot = [one_hot(words,vocab_size) for words in corpus]
onehot[:10]

[[799, 4328, 2081, 1189, 2412, 1523, 4734, 2730, 4177, 3343],
 [789, 3723, 1004, 469, 3679, 640, 3942],
 [3233, 2945, 4428, 960],
 [319, 706, 3195, 3286, 4059, 3772],
 [1370, 3679, 3576, 753, 3943, 2434, 3679, 3568, 4062, 4599],
 [1552,
  4898,
  3078,
  1018,
  3879,
  4172,
  3357,
  111,
  3411,
  921,
  4135,
  633,
  245,
  1333,
  3942],
 [4167, 3143, 2139, 280, 2415, 3467, 3834, 3778, 262, 1889, 3890],
 [4951, 754, 3187, 2299, 2900, 3170, 4172, 716, 262, 1889, 3890],
 [2542, 3582, 2114, 138, 1798, 4414, 309, 265, 4172, 4483],
 [4085, 2039, 3995, 2745, 2436, 4390, 1969, 4511]]

In [31]:
len(onehot)

18285

# Embedding of onehot words

In [32]:
length_df = pd.DataFrame(corpus,columns=['corpus'])

In [34]:
length_df['length'] = length_df['corpus'].str.len()

In [37]:
length_df['length'].max()

299

In [38]:
#Here we have maximum sentence length is 299 so we will take embedding accordingly

In [39]:
length = 30
embedding = pad_sequences(onehot,maxlen=length,padding='pre')
embedding[:10]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,  799, 4328,
        2081, 1189, 2412, 1523, 4734, 2730, 4177, 3343],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,  789, 3723, 1004,  469, 3679,  640, 3942],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0, 3233, 2945, 4428,  960],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,  319,  706, 3195, 3286, 4059, 3772],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0, 1370, 3679,
        3576,  753, 3943, 2434, 

In [59]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 30, 40)            200000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [60]:
X = np.array(embedding)
Y = np.array(y)

In [61]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.33,random_state=42)

In [62]:
#train the model 
model.fit(X_train,y_train,batch_size=64,epochs=20,validation_data=(X_test,y_test))

Epoch 1/20
192/192 [==============================] - 7s 36ms/step - loss: 0.3419 - accuracy: 0.8393 - val_loss: 0.2180 - val_accuracy: 0.9090
Epoch 2/20
192/192 [==============================] - 7s 38ms/step - loss: 0.1407 - accuracy: 0.9459 - val_loss: 0.1886 - val_accuracy: 0.9193
Epoch 3/20
192/192 [==============================] - 8s 42ms/step - loss: 0.0998 - accuracy: 0.9630 - val_loss: 0.2039 - val_accuracy: 0.9138
Epoch 4/20
192/192 [==============================] - 9s 45ms/step - loss: 0.0746 - accuracy: 0.9743 - val_loss: 0.2494 - val_accuracy: 0.9210
Epoch 5/20
192/192 [==============================] - 6s 33ms/step - loss: 0.0501 - accuracy: 0.9833 - val_loss: 0.2686 - val_accuracy: 0.9147
Epoch 6/20
192/192 [==============================] - 6s 33ms/step - loss: 0.0306 - accuracy: 0.9902 - val_loss: 0.3054 - val_accuracy: 0.9140
Epoch 7/20
192/192 [==============================] - 7s 38ms/step - loss: 0.0187 - accuracy: 0.9953 - val_loss: 0.3647 - val_accuracy: 0.9147

In [49]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(vocab_size,embedding_vector_features,input_length=length))
model1.add(Dropout(0.3))
model1.add(LSTM(100))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [50]:
#train the model 
model1.fit(X_train,y_train,batch_size=64,epochs=20,validation_data=(X_test,y_test))

Epoch 1/20
200/200 [==============================] - 7s 34ms/step - loss: 0.3364 - accuracy: 0.8423 - val_loss: 0.2181 - val_accuracy: 0.9114
Epoch 2/20
200/200 [==============================] - 7s 33ms/step - loss: 0.1503 - accuracy: 0.9405 - val_loss: 0.1915 - val_accuracy: 0.9183
Epoch 3/20
200/200 [==============================] - 7s 37ms/step - loss: 0.1121 - accuracy: 0.9594 - val_loss: 0.2063 - val_accuracy: 0.9191
Epoch 4/20
200/200 [==============================] - 7s 36ms/step - loss: 0.0873 - accuracy: 0.9674 - val_loss: 0.2205 - val_accuracy: 0.9171
Epoch 5/20
200/200 [==============================] - 7s 37ms/step - loss: 0.0652 - accuracy: 0.9777 - val_loss: 0.2414 - val_accuracy: 0.9116
Epoch 6/20
200/200 [==============================] - 8s 41ms/step - loss: 0.0491 - accuracy: 0.9837 - val_loss: 0.2637 - val_accuracy: 0.9158
Epoch 7/20
200/200 [==============================] - 8s 39ms/step - loss: 0.0371 - accuracy: 0.9879 - val_loss: 0.3301 - val_accuracy: 0.9156

We can see that after adding dropout we are not getting accuracy more than 90% so we will take different approaches but we can see the entire process of how LSTM cab ne used in text classification

# Now can check the performance of the model 

In [64]:
y_pred=model.predict_classes(X_test)
CM = confusion_matrix(y_test,y_pred)
score = accuracy_score(y_test,y_pred)
print(CM)
print(score)

[[3132  287]
 [ 240 2376]]
0.9126760563380282
